### Load data

In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("./data/train.csv")

### Check the data

In [4]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [5]:
df.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [6]:
df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


### Check for any null values

In [8]:
df.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

#### Having a straight line distance between the pickup and drop off would help us in prediction.  So first create a new feature trip distance from the pickup and drop off longitude & latitude values.

In [9]:
# Obtain distance between two points on earth
# https://en.wikipedia.org/wiki/Great-circle_distance

# Below function gives distance in KM between to points on the earth surface identified by
# their longitude and latitudes (from_long, from_lat) & (to_long, to_lat)
# 0.009 of difference in x co-ordinate = 1.002 km
# 0.009 of difference in y co-ordinate = 1.002 km

from math import sin, cos, acos

def get_distance(from_long, from_lat, to_long, to_lat):
    
    if from_long == to_long and from_lat == to_lat: 
        return 0
    
    v_pi             = 3.1415926; 
    v_earth_radius   = 6378; # Radius of the Earth in km    
    v_from_x_radians = (v_pi / 180) * from_long
    v_from_y_radians = (v_pi / 180) * from_lat
    v_to_x_radians   = (v_pi / 180) * to_long
    v_to_y_radians   = (v_pi / 180) * to_lat

    v_distance = ((acos(sin(v_from_y_radians) * sin(v_to_y_radians) 
                       + (cos(v_from_y_radians) * cos(v_to_y_radians) *  cos(v_from_x_radians - v_to_x_radians)))) 
                  * v_earth_radius)
    return v_distance # Unit in kilometer


In [10]:
get_distance(103.903806, 1.404605, 103.908988, 1.400917)

0.7078785036170167

Calculate the trip distance from the pickup and dropoff longitude & latitudes and add as a new column.

In [11]:
df = pd.concat([df,df.apply(lambda r: get_distance(r['pickup_longitude'], r['pickup_latitude'], r['dropoff_longitude'], r['dropoff_latitude']), axis=1).rename('trip_distance')],axis=1)

In [12]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_distance
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.500167
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1.807491
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6.392114
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.487131
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.189894


#### Instead of predicting drop off at a specific address point, it would be more practical to predict drop offs at a region for example a region of 1 km by 1 km square

As 0.009 difference in latitude / longitude is equivalent to 1.002 km, rounding of the latitude and longitude two decimal places gives us a square of 1 km by 1 km.

In [13]:
df = df.assign(pck_rnd_long = round(df.pickup_longitude,2),
         pck_rnd_lat = round(df.pickup_latitude,2),
         drp_rnd_long = round(df.dropoff_longitude,2),
         drp_rnd_lat = round(df.dropoff_latitude,2) 
         )


In [14]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.500167,-73.98,40.77,-73.96,40.77
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1.807491,-73.98,40.74,-74.00,40.73
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6.392114,-73.98,40.76,-74.01,40.71
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.487131,-74.01,40.72,-74.01,40.71
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.189894,-73.97,40.79,-73.97,40.78


#### Get a pickup time hour rounded to 15 minutes slot and add as a new feature

In [15]:
hrmi = df.pickup_datetime[0][11:16]

In [16]:
def round_to_mins(hrmi, p_i_min):
    hr = int(hrmi[0:2])         
    mi = (int(hrmi[3:5]) // p_i_min) * p_i_min
    return (hr + mi/60)

In [17]:
round_to_mins(df.pickup_datetime[0][11:16],15)

17.25

In [18]:
df['pickup_hrmi'] = df.apply(lambda r: round_to_mins(r['pickup_datetime'][11:16],15), axis=1)

In [19]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat,pickup_hrmi
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.500167,-73.98,40.77,-73.96,40.77,17.25
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1.807491,-73.98,40.74,-74.00,40.73,0.50
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6.392114,-73.98,40.76,-74.01,40.71,11.50
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.487131,-74.01,40.72,-74.01,40.71,19.50
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.189894,-73.97,40.79,-73.97,40.78,13.50


#### Find the weekday of the pickup date and add as a new feature

In [20]:
from datetime import datetime
datetime.strptime('2018-10-05','%Y-%m-%d').weekday()

4

In [21]:
datetime.strptime(df.pickup_datetime[0][0:10],'%Y-%m-%d').weekday()

0

In [22]:
df.head().apply(lambda r: datetime.strptime(r['pickup_datetime'][0:10],'%Y-%m-%d').weekday(),axis=1)

0    0
1    6
2    1
3    2
4    5
dtype: int64

In [23]:
df['pickup_weekday'] = df.apply(lambda r: datetime.strptime(r['pickup_datetime'][0:10],'%Y-%m-%d').weekday(), axis=1)

In [24]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat,pickup_hrmi,pickup_weekday
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.500167,-73.98,40.77,-73.96,40.77,17.25,0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1.807491,-73.98,40.74,-74.00,40.73,0.50,6
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6.392114,-73.98,40.76,-74.01,40.71,11.50,1
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.487131,-74.01,40.72,-74.01,40.71,19.50,2
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.189894,-73.97,40.79,-73.97,40.78,13.50,5


#### Check for trip duration more than 2 hours 

In [54]:
round(df.loc[df['trip_duration'] > 2 * 60 * 60].trip_duration/60/60,0).value_counts()

24.0     1595
23.0      285
2.0       107
3.0        57
22.0       31
4.0        16
8.0        15
10.0       15
17.0       14
21.0       14
7.0        13
6.0        12
9.0        10
11.0        8
18.0        8
12.0        8
16.0        7
15.0        7
19.0        7
20.0        6
5.0         5
14.0        5
13.0        4
980.0       1
619.0       1
569.0       1
539.0       1
Name: trip_duration, dtype: int64

#### Check trip distance

In [57]:
round(df.trip_distance,0).value_counts()

1.0       444335
2.0       356495
3.0       195111
4.0       111663
5.0        71053
6.0        49005
0.0        48782
7.0        32726
9.0        24931
8.0        23229
10.0       21328
11.0       11911
21.0       10357
20.0       10260
12.0        7567
14.0        6531
13.0        6374
19.0        4346
15.0        4096
22.0        4045
16.0        3973
18.0        3401
17.0        3318
23.0        1708
24.0         553
25.0         274
26.0         251
27.0         191
28.0         134
29.0          92
           ...  
576.0          1
320.0          1
55.0           1
892.0          1
173.0          1
117.0          1
115.0          1
113.0          1
215.0          1
105.0          1
102.0          1
98.0           1
255.0          1
94.0           1
91.0           1
86.0           1
84.0           1
83.0           1
315.0          1
1242.0         1
76.0           1
75.0           1
72.0           1
71.0           1
70.0           1
68.0           1
192.0          1
135.0         

In [59]:
df.loc[df['trip_distance'] > 30].count()

id                    575
vendor_id             575
pickup_datetime       575
dropoff_datetime      575
passenger_count       575
pickup_longitude      575
pickup_latitude       575
dropoff_longitude     575
dropoff_latitude      575
store_and_fwd_flag    575
trip_duration         575
trip_distance         575
pck_rnd_long          575
pck_rnd_lat           575
drp_rnd_long          575
drp_rnd_lat           575
pickup_hrmi           575
pickup_weekday        575
dtype: int64

#### Based on above we can now assume that pickups started within 30 KM from the drop off location within past 2 hours are the major contributors to drop off

In [64]:
df.loc[0].pickup_datetime

'2016-03-14 17:24:55'

In [65]:
datetime.strptime(df.loc[0].pickup_datetime,'%Y-%m-%d %H:%M:%S')

datetime.datetime(2016, 3, 14, 17, 24, 55)

In [70]:
datetime.strptime(df.loc[0].pickup_datetime,'%Y-%m-%d %H:%M:%S') - datetime(2016, 3, 13, 17, 23, 55)

datetime.timedelta(1, 60)

In [72]:
from datetime import timedelta

datetime.strptime(df.loc[0].pickup_datetime,'%Y-%m-%d %H:%M:%S') - timedelta(hours=1)

datetime.datetime(2016, 3, 14, 16, 24, 55)

#### Add feature pickup date time in python date time format for ease of calculation

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 18 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
trip_distance         1458644 non-null float64
pck_rnd_long          1458644 non-null float64
pck_rnd_lat           1458644 non-null float64
drp_rnd_long          1458644 non-null float64
drp_rnd_lat           1458644 non-null float64
pickup_hrmi           1458644 non-null float64
pickup_weekday        1458644 non-null int64
dtypes: float64(10), int64(4), object(4)
memory usage

In [75]:
df['pickup_dtm'] = df.apply(lambda r: datetime.strptime(r['pickup_datetime'],'%Y-%m-%d %H:%M:%S'),axis=1)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 19 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
trip_distance         1458644 non-null float64
pck_rnd_long          1458644 non-null float64
pck_rnd_lat           1458644 non-null float64
drp_rnd_long          1458644 non-null float64
drp_rnd_lat           1458644 non-null float64
pickup_hrmi           1458644 non-null float64
pickup_weekday        1458644 non-null int64
pickup_dtm            1458644 non-null datetime64[ns]

In [77]:
df['dropoff_dtm'] = df.apply(lambda r: datetime.strptime(r['dropoff_datetime'],'%Y-%m-%d %H:%M:%S'),axis=1)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 20 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
trip_distance         1458644 non-null float64
pck_rnd_long          1458644 non-null float64
pck_rnd_lat           1458644 non-null float64
drp_rnd_long          1458644 non-null float64
drp_rnd_lat           1458644 non-null float64
pickup_hrmi           1458644 non-null float64
pickup_weekday        1458644 non-null int64
pickup_dtm            1458644 non-null datetime64[ns]

In [95]:
# Function that filters given datframe for pickup_dtm column >= (dtm - hourss) and < dtm
def filter_pickups_for_duration(df, dtm, hours):
    aft = df[df['pickup_dtm'] >= (dtm - timedelta(hours=hours))] 
    return aft[aft['pickup_dtm'] < dtm]

In [98]:
filter_pickups_for_duration(df, datetime(2016,3,14,18,0,0), 1)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat,pickup_hrmi,pickup_weekday,pickup_dtm,dropoff_dtm
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.500167,-73.98,40.77,-73.96,40.77,17.25,0,2016-03-14 17:24:55,2016-03-14 17:32:30
2542,id3758366,2,2016-03-14 17:19:42,2016-03-14 17:26:17,1,-73.993095,40.747917,-74.006340,40.734406,N,395,1.873524,-73.99,40.75,-74.01,40.73,17.25,0,2016-03-14 17:19:42,2016-03-14 17:26:17
9410,id0790528,2,2016-03-14 17:01:41,2016-03-14 17:14:21,1,-73.984245,40.749043,-73.999893,40.734074,N,760,2.125623,-73.98,40.75,-74.00,40.73,17.00,0,2016-03-14 17:01:41,2016-03-14 17:14:21
12042,id2704865,2,2016-03-14 17:16:07,2016-03-14 18:01:40,1,-74.006760,40.705559,-73.980568,40.787540,N,2733,9.389473,-74.01,40.71,-73.98,40.79,17.25,0,2016-03-14 17:16:07,2016-03-14 18:01:40
16124,id1226973,1,2016-03-14 17:52:46,2016-03-14 17:58:44,1,-73.984711,40.759007,-73.991318,40.750095,N,358,1.137711,-73.98,40.76,-73.99,40.75,17.75,0,2016-03-14 17:52:46,2016-03-14 17:58:44
19931,id3325229,1,2016-03-14 17:02:59,2016-03-14 17:24:37,1,-74.000107,40.732910,-73.983337,40.766312,N,1298,3.978011,-74.00,40.73,-73.98,40.77,17.00,0,2016-03-14 17:02:59,2016-03-14 17:24:37
31451,id0069024,2,2016-03-14 17:12:51,2016-03-14 17:13:27,1,-73.951118,40.741631,-73.954269,40.741585,N,36,0.265801,-73.95,40.74,-73.95,40.74,17.00,0,2016-03-14 17:12:51,2016-03-14 17:13:27
32637,id0881970,1,2016-03-14 17:12:16,2016-03-14 17:31:23,1,-73.960800,40.765873,-73.985634,40.778114,N,1147,2.497948,-73.96,40.77,-73.99,40.78,17.00,0,2016-03-14 17:12:16,2016-03-14 17:31:23
32999,id1579078,1,2016-03-14 17:13:17,2016-03-14 17:17:31,1,-73.968628,40.791573,-73.977722,40.782867,N,254,1.235525,-73.97,40.79,-73.98,40.78,17.00,0,2016-03-14 17:13:17,2016-03-14 17:17:31
38351,id2747559,1,2016-03-14 17:37:59,2016-03-14 17:53:14,1,-73.951706,40.774109,-73.982567,40.738621,N,915,4.730501,-73.95,40.77,-73.98,40.74,17.50,0,2016-03-14 17:37:59,2016-03-14 17:53:14


In [119]:
f = filter_pickups_for_duration(df, datetime(2016,3,14,18,0,0), 0.25)

In [120]:
f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 16124 to 1455483
Data columns (total 20 columns):
id                    128 non-null object
vendor_id             128 non-null int64
pickup_datetime       128 non-null object
dropoff_datetime      128 non-null object
passenger_count       128 non-null int64
pickup_longitude      128 non-null float64
pickup_latitude       128 non-null float64
dropoff_longitude     128 non-null float64
dropoff_latitude      128 non-null float64
store_and_fwd_flag    128 non-null object
trip_duration         128 non-null int64
trip_distance         128 non-null float64
pck_rnd_long          128 non-null float64
pck_rnd_lat           128 non-null float64
drp_rnd_long          128 non-null float64
drp_rnd_lat           128 non-null float64
pickup_hrmi           128 non-null float64
pickup_weekday        128 non-null int64
pickup_dtm            128 non-null datetime64[ns]
dropoff_dtm           128 non-null datetime64[ns]
dtypes: datetime64[ns](2)

In [124]:
def filter_pickups_for_distance(df, longitude, latitude, distance):
    tdf = df
    tdf['dist'] = tdf.apply(lambda r: get_distance(r['pickup_longitude'], r['pickup_latitude'], r['dropoff_longitude'], r['dropoff_latitude']), axis=1)
    return tdf[tdf['dist'] < distance]
    

In [114]:
f.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat,pickup_hrmi,pickup_weekday,pickup_dtm,dropoff_dtm,dist
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,1.500167,-73.98,40.77,-73.96,40.77,17.25,0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.500167
2542,id3758366,2,2016-03-14 17:19:42,2016-03-14 17:26:17,1,-73.993095,40.747917,-74.006340,40.734406,N,...,1.873524,-73.99,40.75,-74.01,40.73,17.25,0,2016-03-14 17:19:42,2016-03-14 17:26:17,1.873524
9410,id0790528,2,2016-03-14 17:01:41,2016-03-14 17:14:21,1,-73.984245,40.749043,-73.999893,40.734074,N,...,2.125623,-73.98,40.75,-74.00,40.73,17.00,0,2016-03-14 17:01:41,2016-03-14 17:14:21,2.125623
12042,id2704865,2,2016-03-14 17:16:07,2016-03-14 18:01:40,1,-74.006760,40.705559,-73.980568,40.787540,N,...,9.389473,-74.01,40.71,-73.98,40.79,17.25,0,2016-03-14 17:16:07,2016-03-14 18:01:40,9.389473
16124,id1226973,1,2016-03-14 17:52:46,2016-03-14 17:58:44,1,-73.984711,40.759007,-73.991318,40.750095,N,...,1.137711,-73.98,40.76,-73.99,40.75,17.75,0,2016-03-14 17:52:46,2016-03-14 17:58:44,1.137711


In [127]:
filter_pickups_for_distance(f, -74.006760, 40.705559, 1 ).pickup_dtm.count()

22

In [128]:
ttdf = df.head(100)

In [131]:
ttdf

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,trip_distance,pck_rnd_long,pck_rnd_lat,drp_rnd_long,drp_rnd_lat,pickup_hrmi,pickup_weekday,pickup_dtm,dropoff_dtm,pckup_15m_10k
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,1.500167,-73.98,40.77,-73.96,40.77,17.25,0,2016-03-14 17:24:55,2016-03-14 17:32:30,99
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,1.807491,-73.98,40.74,-74.00,40.73,0.50,6,2016-06-12 00:43:35,2016-06-12 00:54:38,126
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,6.392114,-73.98,40.76,-74.01,40.71,11.50,1,2016-01-19 11:35:24,2016-01-19 12:10:48,95
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,1.487131,-74.01,40.72,-74.01,40.71,19.50,2,2016-04-06 19:32:31,2016-04-06 19:39:40,129
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,1.189894,-73.97,40.79,-73.97,40.78,13.50,5,2016-03-26 13:30:55,2016-03-26 13:38:10,96
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,...,1.100150,-73.98,40.74,-73.99,40.75,22.00,5,2016-01-30 22:01:40,2016-01-30 22:09:03,133
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,...,1.327736,-73.97,40.76,-73.96,40.77,22.50,4,2016-06-17 22:34:59,2016-06-17 22:40:40,124
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,...,5.721260,-73.97,40.80,-73.92,40.76,7.75,5,2016-05-21 07:54:58,2016-05-21 08:20:49,38
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,...,1.311793,-74.00,40.74,-73.99,40.73,23.00,4,2016-05-27 23:12:23,2016-05-27 23:16:38,99
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,...,5.126788,-73.98,40.74,-73.97,40.79,21.75,3,2016-03-10 21:45:01,2016-03-10 22:05:26,126


In [135]:
filter_pickups_for_distance(filter_pickups_for_duration(df, datetime(2016,3,14,17,32,30),0.25), -73.964630, 40.765602,10).pickup_dtm.count()

99

In [136]:
ttdf['pckup_15m_10k'] = ttdf.apply(lambda r: filter_pickups_for_distance(filter_pickups_for_duration(ttdf, r['dropoff_dtm'],0.25), r['dropoff_longitude'], r['dropoff_latitude'],10).pickup_dtm.count() , axis=1)

ValueError: ('Cannot set a frame with no defined index and a value that cannot be converted to a Series', 'occurred at index 2')